In [30]:
# !pip install finta

In [31]:
import pandas as pd
from finta import TA
from datetime import timedelta
import numpy as np
"""
class StockDataProcessor:
      def __init__(self, df, crossover_columns=None):
        self.df = df
        self.crossover_columns = crossover_columns
      def __call__(self):
        self.crossover()
        for crossover_column in self.df.filter(like="_Crossover").columns:
            self.df[f"{crossover_column}_DaysSinceCrossover"] = self.get_days_since_crossover(crossover_column)
            self.df[f"{crossover_column}_Direction"] = self.get_crossover_direction(crossover_column)
        return None
      def crossover(self):
          temp_list = self.crossover_columns.copy()
          for i in temp_list.copy():
              temp_list.remove(i)
              for j in temp_list:
                  self.df[f"{i}_{j}_Crossover"] = self.df[i] / self.df[j]
      def get_crossover(self, crossover_column):
          return self.df[crossover_column]
      def get_crossover_direction(self, crossover_column):
          return np.where(
              (self.df[crossover_column] > 1 ) & (self.df[crossover_column].shift(1) < 1), 1,
              np.where(
                  (self.df[crossover_column] < 1) & (self.df[crossover_column].shift(1) > 1), -1, 0,
              )
          )
      def get_days_since_crossover(self, crossover_column):
          days_since_crossover = pd.Series(np.zeros(len(self.df)), index=self.df.index)
          crossover_direction = self.get_crossover_direction(crossover_column)

          day = 0
          for x in range(len(crossover_column)):
              if crossover_direction[x] == 1 or crossover_direction[x] == -1:
                  day = 0
              else:
                  day += 1
              days_since_crossover[x] = day

          return days_since_crossover

"""

'\nclass StockDataProcessor:\n      def __init__(self, df, crossover_columns=None):\n        self.df = df\n        self.crossover_columns = crossover_columns\n      def __call__(self):\n        self.crossover()\n        for crossover_column in self.df.filter(like="_Crossover").columns:\n            self.df[f"{crossover_column}_DaysSinceCrossover"] = self.get_days_since_crossover(crossover_column)\n            self.df[f"{crossover_column}_Direction"] = self.get_crossover_direction(crossover_column)\n        return None\n      def crossover(self):\n          temp_list = self.crossover_columns.copy()\n          for i in temp_list.copy():\n              temp_list.remove(i)\n              for j in temp_list:\n                  self.df[f"{i}_{j}_Crossover"] = self.df[i] / self.df[j]\n      def get_crossover(self, crossover_column):\n          return self.df[crossover_column]\n      def get_crossover_direction(self, crossover_column):\n          return np.where(\n              (self.df[crosso

In [32]:
df = pd.read_csv("/content/tickers.csv")
df = df[df.tic == "^TNX"]
df = df.drop(["Unnamed: 0", "tic", "day"], axis=1)
df = df.rename(columns = {'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
df['SMA5'] = TA.SMA(df, 5)
# df['SMA20'] = TA.SMA(df, 20)
# df['EMA5'] = TA.EMA(df, 5)
# df['EMA20'] = TA.EMA(df, 20)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_values('date', ascending=True, inplace=True)

In [33]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [34]:
import numpy as np
import pandas as pd

class StockDataProcessor_1:
    """
    A class for processing stock data, calculating crossover events, and providing additional analysis.

    Args:
        df (pd.DataFrame): The input DataFrame containing stock data.
        crossover_columns (list): A list of column names for which crossover events are to be calculated.
        include_columns (bool): If True, all columns are included in the resulting DataFrame.
        days_since_crossover_suffix (str): The suffix for 'DaysSinceCrossover' column names.
        direction_suffix (str): The suffix for 'Direction' column names.
        crossover_suffix (str): The suffix for 'Crossover' column names.

    Returns:
        StockDataProcessor: An instance of the StockDataProcessor class.

    Example Usage:
        processor = StockDataProcessor(df, crossover_columns=["Close", "SMA5"], include_columns=True)
        resulting_df = processor()
        resulting_df.fillna(0, inplace=True)
        resulting_df.head(50)
    """

    def __init__(self, df, crossover_columns=None, include_columns=False,
                 days_since_crossover_suffix='_DaysSinceCrossover',
                 direction_suffix='_Direction', crossover_suffix='_Crossover'):
        """
        Initialize the StockDataProcessor with the input DataFrame and parameters.

        Args:
            df (pd.DataFrame): The input DataFrame containing stock data.
            crossover_columns (list): A list of column names for which crossover events are to be calculated.
            include_columns (bool): If True, all columns are included in the resulting DataFrame.
            days_since_crossover_suffix (str): The suffix for 'DaysSinceCrossover' column names.
            direction_suffix (str): The suffix for 'Direction' column names.
            crossover_suffix (str): The suffix for 'Crossover' column names.
        """
        self.df = df.copy()
        self.crossover_columns = crossover_columns
        self.include_columns = include_columns
        self.days_since_crossover_suffix = days_since_crossover_suffix
        self.direction_suffix = direction_suffix
        self.crossover_suffix = crossover_suffix

    def __call__(self):
        """
        Process the stock data, calculate crossover events, and return the resulting DataFrame.

        Returns:
            pd.DataFrame: The resulting DataFrame with crossover-related columns.
        """
        processed_df = self.crossover()
        if self.include_columns:
            return processed_df
        else:
            return self.df

    def crossover(self):
        """
        Calculate crossover events for the specified columns in the DataFrame.

        Returns:
            pd.DataFrame: The DataFrame with crossover events added.
        """
        temp_list = self.crossover_columns.copy()
        for i in temp_list.copy():
            temp_list.remove(i)
            for j in temp_list:
                self.df[f"{i}_{j}{self.crossover_suffix}"] = self.df[i] / self.df[j]

        for crossover_column in self.df.filter(like=self.crossover_suffix).columns:
            self.df[f"{crossover_column}{self.days_since_crossover_suffix}"] = self.get_days_since_crossover(crossover_column)
            self.df[f"{crossover_column}{self.direction_suffix}"] = self.get_crossover_direction(crossover_column)

        return self.df

    def get_crossover(self, crossover_column):
        """
        Get the crossover values for a specific column.

        Args:
            crossover_column (str): The name of the crossover column to retrieve.

        Returns:
            pd.Series: A Series containing crossover values for the specified column.
        """
        return self.df[crossover_column]

    def get_crossover_direction(self, crossover_column):
        """
        Calculate the direction of crossover events for a specific column.

        Args:
            crossover_column (str): The name of the crossover column to calculate direction for.

        Returns:
            np.ndarray: An array indicating the direction of crossover events.
        """
        return np.where(
            (self.df[crossover_column] > 1) & (self.df[crossover_column].shift(1) < 1), 1,
            np.where(
                (self.df[crossover_column] < 1) & (self.df[crossover_column].shift(1) > 1), -1, 0,
            )
        )

    def get_days_since_crossover(self, crossover_column):
        """
        Calculate the number of days since a crossover event occurred for a specific column.

        Args:
            crossover_column (str): The name of the crossover column to calculate days since crossover for.

        Returns:
            pd.Series: A Series containing the number of days since a crossover event.
        """
        days_since_crossover = pd.Series(np.zeros(len(self.df)), index=self.df.index)
        crossover_direction = self.get_crossover_direction(crossover_column)

        day = 0
        for x in range(len(self.df)):
            if crossover_direction[x] == 1 or crossover_direction[x] == -1:
                day = 0
            else:
                day += 1
            days_since_crossover[x] = day

        return days_since_crossover

In [35]:
import numpy as np
import pandas as pd

class StockDataProcessor_2:
    """
    A class for processing stock data, calculating crossover events, and providing additional analysis.

    Explanation
    -----------
    This class processes stock data, calculates crossover events, and provides additional analysis.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing stock data.
    crossover_columns : list
        A list of column names for which crossover events are to be calculated.
    include_columns : bool
        If True, all columns are included in the resulting DataFrame.
    days_since_crossover_suffix : str
        The suffix for 'DaysSinceCrossover' column names.
    direction_suffix : str
        The suffix for 'Direction' column names.
    crossover_suffix : str
        The suffix for 'Crossover' column names.

    Returns
    -------
    StockDataProcessor
        An instance of the StockDataProcessor class.

    Example
    -------
    processor = StockDataProcessor(df, crossover_columns=["Close", "SMA5"], include_columns=True)
    resulting_df = processor()
    resulting_df.fillna(0, inplace=True)
    resulting_df.head(50)
    """

    def __init__(self, df, crossover_columns=None, include_columns=False,
                 days_since_crossover_suffix='_DaysSinceCrossover',
                 direction_suffix='_Direction', crossover_suffix='_Crossover'):
        """
        Initialize the StockDataProcessor with the input DataFrame and parameters.

        Parameters
        ----------
        df : pd.DataFrame
            The input DataFrame containing stock data.
        crossover_columns : list
            A list of column names for which crossover events are to be calculated.
        include_columns : bool
            If True, all columns are included in the resulting DataFrame.
        days_since_crossover_suffix : str
            The suffix for 'DaysSinceCrossover' column names.
        direction_suffix : str
            The suffix for 'Direction' column names.
        crossover_suffix : str
            The suffix for 'Crossover' column names.
        """
        self.df = df.copy()
        self.crossover_columns = crossover_columns
        self.include_columns = include_columns
        self.days_since_crossover_suffix = days_since_crossover_suffix
        self.direction_suffix = direction_suffix
        self.crossover_suffix = crossover_suffix

    def __call__(self):
        """
        Process the stock data, calculate crossover events, and return the resulting DataFrame.

        Returns
        -------
        pd.DataFrame
            The resulting DataFrame with crossover-related columns.
        """
        processed_df = self._crossover()
        if self.include_columns:
            return processed_df
        else:
            return self.df

    def _crossover(self):
        """
        Calculate crossover events for the specified columns in the DataFrame.

        Returns
        -------
        pd.DataFrame
            The DataFrame with crossover events added.
        """
        temp_list = self.crossover_columns.copy()

        for i in temp_list.copy():
            temp_list.remove(i)
            for j in temp_list:
                crossover_temp = self.df[i] / self.df[j]
                crossover_col_name = f"{i}_{j}{self.crossover_suffix}"
                self.df[crossover_col_name] = crossover_temp

        for crossover_column in self.df.filter(like=self.crossover_suffix).columns:
            days_since_crossover_col = f"{crossover_column}{self.days_since_crossover_suffix}"
            direction_col = f"{crossover_column}{self.direction_suffix}"

            self.df[days_since_crossover_col] = self._get_days_since_crossover(crossover_column)
            self.df[direction_col] = self._get_crossover_direction(crossover_column)

        return self.df

    def _get_crossover_direction(self, crossover_column):
        """
        Calculate the direction of crossover events for a specific column.

        Parameters
        ----------
        crossover_column : str
            The name of the crossover column to calculate direction for.

        Returns
        -------
        np.ndarray
            An array indicating the direction of crossover events.
        """
        condition_1 = (self.df[crossover_column] > 1) & (self.df[crossover_column].shift(1) < 1)
        condition_2 = (self.df[crossover_column] < 1) & (self.df[crossover_column].shift(1) > 1)

        temp_result = np.where(condition_1, 1, np.where(condition_2, -1, 0))
        return temp_result

    def _get_days_since_crossover(self, crossover_column):
        """
        Calculate the number of days since a crossover event occurred for a specific column.

        Parameters
        ----------
        crossover_column : str
            The name of the crossover column to calculate days since crossover for.

        Returns
        -------
        pd.Series
            A Series containing the number of days since a crossover event.
        """
        days_since_crossover = pd.Series(np.zeros(len(self.df)), index=self.df.index)
        crossover_direction = self._get_crossover_direction(crossover_column)

        day = 0
        for x in range(len(self.df)):
            if crossover_direction[x] == 1 or crossover_direction[x] == -1:
                day = 0
            else:
                day += 1
            days_since_crossover[x] = day

        return days_since_crossover

In [36]:
# use docsting like following
"""
Explanation
Paramaters
----------
name: str
  parameter explanation
ta : list
  parameter explanation
Returns
-------
df: pd.dataFrame()
  return explanation
Example
-------
Notes
-------
"""

'\nExplanation\nParamaters\n----------\nname: str\n  parameter explanation\nta : list\n  parameter explanation\nReturns\n-------\ndf: pd.dataFrame()\n  return explanation\nExample\n-------\nNotes\n-------\n'

In [37]:
df1 = df.copy()
df1 = df1.drop(["Open", "High", "Low", "Volume"], axis=1)

In [38]:
df2 = df1.copy()
# Example usage
processor_2 = StockDataProcessor_2(df2, crossover_columns=["Close", "SMA5"], include_columns=False, )
                               # days_since_crossover_suffix="Day", direction_suffix="Dir", crossover_suffix="Cross")
resulting_df_2 = processor_2()
resulting_df_2.fillna(0, inplace=True)
resulting_df_2.head(50)

,Close,SMA5,Close_SMA5_Crossover,Close_SMA5_Crossover_DaysSinceCrossover,Close_SMA5_Crossover_Direction
date,,,,,
2000-01-04,6.485,0.0000,0.000000,1.0,0
2000-01-05,6.599,0.0000,0.000000,2.0,0
2000-01-06,6.549,0.0000,0.000000,3.0,0
2000-01-07,6.504,0.0000,0.000000,4.0,0
2000-01-11,6.664,6.5602,1.015823,5.0,0
2000-01-12,6.696,6.6024,1.014177,6.0,0
2000-01-13,6.618,6.6062,1.001786,7.0,0
2000-01-14,6.674,6.6312,1.006454,8.0,0
2000-01-18,6.748,6.6800,1.010180,9.0,0


In [39]:
# Example usage
processor_1 = StockDataProcessor_1(df1, crossover_columns=["Close", "SMA5"], include_columns=False, )
                               # days_since_crossover_suffix="Day", direction_suffix="Dir", crossover_suffix="Cross")
resulting_df_1 = processor_1()
resulting_df_1.fillna(0, inplace=True)
resulting_df_1.head(50)

,Close,SMA5,Close_SMA5_Crossover,Close_SMA5_Crossover_DaysSinceCrossover,Close_SMA5_Crossover_Direction
date,,,,,
2000-01-04,6.485,0.0000,0.000000,1.0,0
2000-01-05,6.599,0.0000,0.000000,2.0,0
2000-01-06,6.549,0.0000,0.000000,3.0,0
2000-01-07,6.504,0.0000,0.000000,4.0,0
2000-01-11,6.664,6.5602,1.015823,5.0,0
2000-01-12,6.696,6.6024,1.014177,6.0,0
2000-01-13,6.618,6.6062,1.001786,7.0,0
2000-01-14,6.674,6.6312,1.006454,8.0,0
2000-01-18,6.748,6.6800,1.010180,9.0,0


In [40]:
# Compare DataFrames using the equals method
if resulting_df_1.equals(resulting_df_2):
    print("DataFrames are equal.")
else:
    print("DataFrames are not equal.")

DataFrames are equal.
